In [1]:
from Player import Player
from Trainer import Trainer
from TetrisModel import TetrisModel
from Pretrainer import Pretrainer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import pickle
import glob
import time

In [44]:
piece_dim = 8
key_dim = 12
depth = 16
gamma = 0.99
lam = 0.95
temperature = 1.0

In [3]:
# Use lambda instead of gamma to immitate shape of gae without value predictions
# pretrainer = Pretrainer(gamma=lam)

In [4]:
# players_data = pretrainer._load_data()

In [5]:
# pretrainer._load_dset(players_data)

In [120]:
# max_len = pretrainer._max_len
max_len = 10

In [121]:
# gt_dset = pretrainer._cache_dset()

In [122]:
agent = TetrisModel(piece_dim=piece_dim,
                    key_dim=key_dim,
                    depth=depth,
                    num_heads=4,
                    num_layers=4,
                    max_length=max_len,
                    out_dim=key_dim)

In [123]:
# agent_optimizer = keras.optimizers.Adam(learning_rate=1e-5)
agent.compile(optimizer=agent_optimizer)

In [124]:
logits = agent((tf.random.uniform((1, 28, 10, 1)),
                tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                tf.random.uniform((1, max_len), minval=0, maxval=key_dim, dtype=tf.int32)))
agent.summary(), tf.shape(logits)

Model: "tetris_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_27 (Sequential)  (1, 70, 16)               4800      
                                                                 
 seq_embedding_6 (SeqEmbeddi  multiple                 128       
 ng)                                                             
                                                                 
 seq_embedding_7 (SeqEmbeddi  multiple                 192       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  9776      
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  9776      
                                                                 
 piece_dec_2 (DecoderLayer)  multiple               

(None, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 10, 12])>)

In [125]:
critic = TetrisModel(piece_dim=piece_dim,
                     key_dim=key_dim,
                     depth=depth,
                     num_heads=4,
                     num_layers=4,
                     max_length=max_len,
                     out_dim=1)

In [126]:
# critic_optimizer = keras.optimizers.Adam(learning_rate=1e-5)
critic.compile(optimizer=critic_optimizer)

In [127]:
values = critic((tf.random.uniform((1, 28, 10, 1)),
                 tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                 tf.random.uniform((1, max_len), minval=0, maxval=key_dim, dtype=tf.int32)))
critic.summary(), tf.shape(values)

Model: "tetris_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_36 (Sequential)  (1, 70, 16)               4800      
                                                                 
 seq_embedding_8 (SeqEmbeddi  multiple                 128       
 ng)                                                             
                                                                 
 seq_embedding_9 (SeqEmbeddi  multiple                 192       
 ng)                                                             
                                                                 
 piece_dec_0 (DecoderLayer)  multiple                  9776      
                                                                 
 piece_dec_1 (DecoderLayer)  multiple                  9776      
                                                                 
 piece_dec_2 (DecoderLayer)  multiple               

(None, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 1, 10,  1])>)

In [128]:
agent.load_weights('agent_weights_small/agent_finetuned_4712')
critic.load_weights('critic_weights_small/critic_finetuned_4712')
# with open('agent_weights_small/optimizer.pkl', 'rb') as f:
#     weight_values = pickle.load(f)
# agent_optimizer.set_weights(weight_values)

In [15]:
ref_agent = TetrisModel(piece_dim=piece_dim,
                        key_dim=key_dim,
                        depth=depth,
                        num_heads=4,
                        num_layers=4,
                        max_length=max_len,
                        out_dim=key_dim)

In [16]:
logits = ref_agent((tf.random.uniform((1, 28, 10, 1)),
                    tf.random.uniform((1, 7), minval=0, maxval=8, dtype=tf.int32),
                    tf.random.uniform((1, max_len), minval=0, maxval=key_dim, dtype=tf.int32)))

In [43]:
ref_agent.load_weights('agent_weights_small/agent')

In [18]:
epochs = 10

In [19]:
# actor_losses, critic_losses, accs = pretrainer.train(agent, critic, gt_dset, epochs)

In [20]:
# plt.plot(actor_losses)
# plt.plot(critic_losses)
# plt.plot(accs)

In [21]:
# if 'y' in input('YOU SURE?????'):
#     ref_agent.save_weights('agent_weights_small/agent_reference')

In [24]:
%matplotlib qt

In [129]:
trainer = Trainer(agent=agent,
                  critic=critic,
                  ref_model=ref_agent,
                  max_len=max_len,
                  gamma=gamma,
                  lam=lam,
                  temperature=temperature,
                  max_episode_steps=100,
                  buffer_cap=500)

critic_loss,▂▂▁▂▁▂▂▄▄▃▂▁▄▂▃▅▆▃▅▂█▄▄▃▃▆▃▅▄▂▂▄▂▅▅▃▃▅▃▆
kl_penalty,▃▅▁█▅▂▄▄▆▅▇▄▇▅▃▂▅▆▃▃▆▅▇▅▆▅▆▄▂▅▄▇▇▆▄▅▆▅█▇
ppo_loss,▅▂▃▆▇▂▄▃▅▅▆▄▇▆▄▃▄▆▂▅▂▁▄█▄▇█▆▅▄▃▆▅▆▇▇▄▂▁▄
reward,▂▅▂▂▃▆▇▆▁▃▃▄▁▁▄▄▁▆▄▃▅█▁▅▄▃▇▇▅█▄▂▄▇▄▄▇▆█▆
reward_per_piece,▄▃▁▃▄▃▂▁▅▃▂▄▆▄▄▄▆▄▅▄▅▇▅▂▂▇▂▅▅▅▄▅▂▄█▅▄▆▄▄
unclipped_proportion,▁▃▃▂▅▄▆▄▄▂▄▄▁▃▆▄▄▅▄▅▃▄▃▅▆▃▆▄▅▄▃▄▆▅▄█▅▄▄▆
critic_loss,0.77691
kl_penalty,0.05864
ppo_loss,0.07835
reward,7.25
reward_per_piece,0.0725


In [130]:
trainer.fill_replay_buffer()

Done filling replay buffer

In [117]:
while True:
    trainer.train(gens=100, train_steps=5, training_actor=True)
    agent.save_weights(f'agent_weights_small/agent_finetuned_{trainer.wandb_run.step}')
    critic.save_weights(f'critic_weights_small/critic_finetuned_{trainer.wandb_run.step}')

PPO Loss: 0.08	|	KL Penalty: 0.06	|	Critic Loss: 0.78	|	

KeyboardInterrupt: 

In [118]:
if 'y' in input('YOU SURE?????'):
    agent.save_weights(f'agent_weights_small/agent_finetuned_{trainer.wandb_run.step}')
    critic.save_weights(f'critic_weights_small/critic_finetuned_{trainer.wandb_run.step}')
    
    # symbolic_weights = getattr(agent.optimizer, 'weights')
    # weight_values = K.batch_get_value(symbolic_weights)
    # with open('agent_weights_small/optimizer.pkl', 'wb') as f:
    #     pickle.dump(weight_values, f)

YOU SURE????? y


In [112]:
episode_data = trainer.player.run_episode(agent, critic, max_steps=100, greedy=True, renderer=trainer.renderer)

In [113]:
episode_boards, episode_pieces, episode_inputs, episode_probs, episode_values, episode_rewards = episode_data

In [114]:
episode_advantages, episode_returns = trainer._compute_gae(episode_values, episode_rewards, trainer.gamma, trainer.lam)

In [115]:
fig, ax = plt.subplots()
ax.plot(episode_returns, label='Returns')
ax.plot(episode_values, label='Values')
ax.legend()
tf.reduce_sum(episode_rewards)

<tf.Tensor: shape=(), dtype=float32, numpy=5.749999>

In [116]:
fig, ax = plt.subplots()
ax.plot(episode_rewards, label='Returns')
ax.plot(episode_advantages, label='Advantages')
ax.legend()